In [2]:
import pandas as pd
import numpy as np
import numpy_financial as npf
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
from numpy import *

from ggplot import *
from scipy.stats import logistic
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import *
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
import graphviz
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVC, SVR
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


from linearmodels.iv import IV2SLS
from scipy import stats
from scipy.stats import trim_mean #trimmed mean
from scipy.stats import * #chi2_contingency로 카이제곱통계량계산
from scipy.stats import norm

import keyword
import random

#open api 불러올 때 필요한 것
from urllib.request import urlopen #request package 설치
from bs4 import BeautifulSoup #bs4 package 설치

# 그래프에서 한글 깨지지 않기 위헤 사용함
from matplotlib import rcParams
from matplotlib import font_manager,rc

rcParams['axes.unicode_minus']=False
plt.rcParams['font.family']='Malgun Gothic'
font_name=font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
rc('font',family=font_name)

import FinanceDataReader as fdr
import plotly.graph_objects as go
import plotly.subplots as ms
import math



# Lecture 11

In [3]:
df=pd.read_csv('c:/temp/education2020.csv')
df1=df[['income','age','education_year']]
sex_dummies=pd.get_dummies(df['sex'],prefix='s')

In [4]:
# job 9개 범주를 가지고 있다. 9개 더미변수를 만든다. (A는 없애고 하기)
df=df.loc[df['job']!='A']
job_dummies=pd.get_dummies(df['job'],prefix='j')
job_dummies

,j_1,j_2,j_3,j_4,j_5,j_6,j_7,j_8,j_9
0,0,0,1,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0
4,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
18059,0,0,0,0,0,0,0,0,0
18060,0,0,1,0,0,0,0,0,0
18061,0,0,0,0,0,0,0,0,0
18062,0,1,0,0,0,0,0,0,0


In [5]:
df2=pd.concat([df1,sex_dummies,job_dummies],axis=1)
df2.head()

,income,age,education_year,s_1,s_2,j_1,j_2,j_3,j_4,j_5,j_6,j_7,j_8,j_9
0,6593,34,16,1,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,17720,45,21,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,908,73,6,0,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,2748,58,12,1,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1015,27,12,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
edu_model2=ols('income~age+education_year',data=df2).fit()
print(edu_model2.summary())

                            OLS Regression Results                            
Dep. Variable:                 income   R-squared:                       0.164
Model:                            OLS   Adj. R-squared:                  0.164
Method:                 Least Squares   F-statistic:                     1773.
Date:                Thu, 13 Jun 2024   Prob (F-statistic):               0.00
Time:                        20:27:50   Log-Likelihood:            -1.7980e+05
No. Observations:               18064   AIC:                         3.596e+05
Df Residuals:                   18061   BIC:                         3.596e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept        322.1142    277.944      1.

In [7]:
# 더미변수 포함한 regression 해보기
edu_model3=ols('income~age+education_year+s_1+j_2+j_3+j_4+j_5+j_6+j_7+j_8+j_9',data=df2).fit()
print(edu_model3.summary())

                            OLS Regression Results                            
Dep. Variable:                 income   R-squared:                       0.228
Model:                            OLS   Adj. R-squared:                  0.228
Method:                 Least Squares   F-statistic:                     484.8
Date:                Thu, 13 Jun 2024   Prob (F-statistic):               0.00
Time:                        20:27:50   Log-Likelihood:            -1.7883e+05
No. Observations:               18039   AIC:                         3.577e+05
Df Residuals:                   18027   BIC:                         3.578e+05
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept      -2686.6412    304.369     -8.

In [8]:
# 더미변수 회귀분석할 때는, 하나 범주 빼고 regression 하면됨

In [9]:
# s_1: 1971 (양수)
# 남자가 여자(빠진 더미)보다 돈 더 많이 번다

In [10]:
# j_2: 4115
# j_1 보다 4115만큼 돈을 더 벌고 있구나

# Lecture 12 회귀분석과 머신러닝(1)
## 1 머신러닝을 이용한 회귀분석
## 1.1 기계학습
## 1.4 경사하강법
## 2 Tensor flow
- tensor: 숫자를 저장하는 방식, 자료구조를 tensor로 표현한다, 보통 차원을 의미
- scalar는 0차원, 벡터는 1차원, 행렬은 2차원

In [11]:
pip install tensorflow

In [12]:
import tensorflow as tf

In [13]:
df=pd.read_csv('c:/temp/HPI_SEOUL.csv',encoding='CP949')
df.head()

,YYM1,CITY,CITY_CODE,GU_NAME,GU_CODE2,API_TT0,BaseRate,MoRate,FREQ,A_PSUM,...,LTV,DTI,MV_APT,JAEGO,BUNYANG,N_BUNYANG,IND_INDEX,JTP,LOAN_T,MON
0,200603,서울특별시,11,종로중구,1111,75.245608,4.00,5.46,71,2.559029,...,42.912677,93.255607,4.891421,110,0,277,109.907,48.372,104.381241,3
1,200606,서울특별시,11,종로중구,1111,81.009183,4.25,5.48,58,1.774481,...,43.944172,94.827586,4.891421,0,0,277,99.469,47.454,107.564252,6
2,200609,서울특별시,11,종로중구,1111,81.208640,4.50,5.77,90,2.898839,...,44.425621,94.000000,4.891421,0,0,14,123.203,47.670,110.666410,9
3,200612,서울특별시,11,종로중구,1111,88.068940,4.50,5.88,99,3.181162,...,50.340131,95.151515,4.891421,0,136,6,128.718,43.811,101.843742,12
4,200703,서울특별시,11,종로중구,1111,86.507762,4.50,6.20,68,1.955622,...,50.951451,48.890836,6.083816,0,112,0,110.893,42.987,107.877226,3


In [15]:
# LTV: Loan to value : 집값 대비 부채
# DTI: Debt to income ratio : 소득 대비 부채가 몇퍼센트인지
# DSR: ...
# 가장 최근 규제가 무엇인지

In [16]:
a=tf.constant(12)
a

<tf.Tensor: shape=(), dtype=int32, numpy=12>

In [17]:
# constant, variable 등으로 가져올 수 있다

In [18]:
b=tf.constant([1,2,3])
b

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 2, 3])>

In [19]:
# b는 벡터, a는 스칼라가 된다
# shape=(3,): 행의 갯수가 3개라는 뜻

In [20]:
a.ndim

0

In [21]:
# a는 0차원

In [22]:
b.ndim

1

In [23]:
# b는 1차원

In [24]:
a.shape

TensorShape([])

In [25]:
# a는 0차원

In [26]:
b.shape

TensorShape([3])

In [27]:
# b는 3개의 행으로 구성된 벡터

In [29]:
c=tf.constant([[1,2,3],[4,5,6]])
c

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]])>

In [30]:
# shape=(2,3): 행렬의 dimension이 2x3이라는 뜻 가로 2 세로3

In [31]:
c.ndim

2

In [32]:
c.shape

TensorShape([2, 3])

In [33]:
tf.constant([2,3]) # 2개의 값이 있는 1차원의 텐서 구조라는 뜻

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([2, 3])>

In [36]:
# 3차원의 텐서구조 만들기
d=tf.constant([[[1,2,3],[4,5,6],[7,8,9]]])

In [37]:
d.ndim

3

In [38]:
d.shape

TensorShape([1, 3, 3])

In [40]:
# TensorShape([1, 3, 3])
# 뒤에 3,3: 행의 갯수, 열 갯수
# 앞에 1: depth, 행렬의 층
# 3x3 행렬이 하나 있다는 뜻

In [41]:
a=tf.constant(np.arange(1,13)) # np.arange: 1부터 12까지 연속적 숫자 , 벡터임
a.ndim

1

In [42]:
a

<tf.Tensor: shape=(12,), dtype=int32, numpy=array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])>

In [43]:
# reshape: 다른 텐서구조로 바꾸자
# 2x6구조로 바꾸자
b=tf.reshape(a,[2,6])

In [44]:
b

<tf.Tensor: shape=(2, 6), dtype=int32, numpy=
array([[ 1,  2,  3,  4,  5,  6],
       [ 7,  8,  9, 10, 11, 12]])>

In [45]:
# a를 3차원으로 바꿔보자
# 2x3 행렬을 겹겹이(2층) 쌓아보자
c=tf.reshape(a,[2,2,3])
c.ndim

3

In [46]:
c

<tf.Tensor: shape=(2, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]]])>

In [47]:
# 깊이가 2인 2x3 행렬

#### part2

In [48]:
# tensor 연산
x=tf.constant([1,2,3,4,5,6],shape=[2,3])
x

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]])>

In [49]:
y=tf.constant([1,2,3,4,5,6],shape=[2,3])
y

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]])>

In [50]:
tf.math.subtract(x,y)

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[0, 0, 0],
       [0, 0, 0]])>

In [51]:
tf.math.add(x,y)

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ 2,  4,  6],
       [ 8, 10, 12]])>

In [53]:
tf.math.multiply(x,y)

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ 1,  4,  9],
       [16, 25, 36]])>

In [54]:
# 위에는 element by element 곱셈

In [55]:
# 행렬 곱셈 matmul
y1=tf.transpose(y)
tf.matmul(x,y1)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[14, 32],
       [32, 77]])>

### 텐서로 미분하기

In [56]:
# 미분.. x가 움직일 수 있는 여지 있음 -> x를 Variable로 정의
x=tf.Variable(2.0)

In [57]:
with tf.GradientTape() as t:
    y=x**3
    dy_dx=t.gradient(y,x) # 미분해라
print(dy_dx)

tf.Tensor(12.0, shape=(), dtype=float32)


In [58]:
# 미분한 값 : 12 로 나옴

In [59]:
# 미지수가 두개인  경우 텐서에서 미분하기

In [62]:
# y=f(w1, w2)=3*(w1**2)+2*(w1)*(w2)
w1=tf.Variable(1.0) # 1.0에서 시작
w2=tf.Variable(1.0) # 1.0에서 시작

with tf.GradientTape() as t:
    y=3*(w1**2)+2*w1*w2
    grad=t.gradient(y,[w1,w2])
print(grad)

[<tf.Tensor: shape=(), dtype=float32, numpy=8.0>, <tf.Tensor: shape=(), dtype=float32, numpy=2.0>]


In [63]:
# numpy=8.0, numpy=2.0

## 3 ML을 이용한 회귀분석 원리

In [64]:
x_data=[1,2,3,4,5]
W=tf.Variable(2.0)
b=tf.Variable(0.5)

hypothesis=W*x_data+b
print(hypothesis)

tf.Tensor([ 2.5  4.5  6.5  8.5 10.5], shape=(5,), dtype=float32)


In [65]:
# 2.5  4.5  6.5  8.5 10.5: 이게 알아낸 가설의 값
# 이 가설의 값과, y값이 얼마나 가까운지 측정하는게 cost function